# GeekBrains
## Машинное обучение в бизнесе
## ДЗ Урока 4 (Uplift-моделирование)
### Виталий Казанцев

#### Т/з
1. скачать набор данных маркетинговых кампаний отсюда https://www.kaggle.com/davinwijaya/customer-retention
2. там поле conversion - это целевая переменная, а offer - коммуникация. Переименовать поля (conversion -> target, offer -> treatment) и привести поле treatment к бинарному виду (1 или 0, т.е было какое-то предложение или нет) - значение No Offer означает отсутствие коммуникации, а все остальные - наличие.
3. сделать разбиение набора данных не тренировочную и тестовую выборки
4. сделать feature engineering на ваше усмотрение (допускается свобода выбора методов)
5. провести uplift-моделирование 3 способами: одна модель с признаком коммуникации (S learner), модель с трансформацией таргета (трансформация классов п. 2. 1) и вариант с двумя независимыми моделями
6. в конце вывести единую таблицу сравнения метрик uplift@10%, uplift@20% этих 3 моделей
7. построить модель UpliftTreeClassifier и попытаться описать словами полученное дерево
8. (опционально) для модели S learner (модель с дополнительным признаком коммуникации) построить зависимость таргета (конверсии - поле conversion) от значения uplift: 1) сделать прогноз и получить uplift для тестовой выборки 2) отсортировать тестовую выборку по uplift по убыванию 3) разбить на децили (pandas qcut вам в помощь) 4) для каждого дециля посчитать среднюю conversion
9. (опционально) построить модель UpliftRandomForestClassifier и попытаться описать словами полученное дерево

__Задание 1__  
скачать набор данных маркетинговых кампаний отсюда https://www.kaggle.com/davinwijaya/customer-retention

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import numpy as np

from sklearn.model_selection import train_test_split

from sklift.metrics import uplift_at_k
from sklift.viz import plot_uplift_preds
from sklift.models import SoloModel, ClassTransformation, TwoModels

from causalml.inference.tree import UpliftTreeClassifier, UpliftRandomForestClassifier
from causalml.inference.tree import uplift_tree_string, uplift_tree_plot

from catboost import CatBoostClassifier

%matplotlib inline

ValueError: sklearn.tree._tree.TreeBuilder size changed, may indicate binary incompatibility. Expected 80 from C header, got 72 from PyObject

In [ ]:
df = pd.read_csv('data.csv')
df.info()

In [ ]:
df.sample(5)

In [ ]:
df.shape

__Задание 2__  
там поле conversion - это целевая переменная, а offer - коммуникация. Переименовать поля (conversion -> target, offer -> treatment) и привести поле treatment к бинарному виду (1 или 0, т.е было какое-то предложение или нет) - значение No Offer означает отсутствие коммуникации, а все остальные - наличие.

In [ ]:
df.rename(columns={'conversion': 'target', 'offer': 'treatment'}, inplace=True)
df.info()

In [ ]:
df.loc[(df.treatment != 'No Offer'), 'treatment'] = 1
df.loc[(df.treatment == 'No Offer'), 'treatment'] = 0
df = df.astype({'treatment': np.int8})
df.info()

In [ ]:
df.sample(5)

__Задание 3__  
сделать разбиение набора данных не тренировочную и тестовую выборки

In [ ]:
X_train, X_test, y_train, y_test, treat_train, treat_test = train_test_split(df.drop(columns=['target']), 
                                                            df['target'],
                                                            df['treatment'],
                                                            random_state=0)

X_train.shape, X_test.shape, y_train.shape, y_test.shape, treat_train.shape

__Задание 4__  
сделать feature engineering на ваше усмотрение (допускается свобода выбора методов)

Разделим пользователей по объему расходов(expenses)

In [ ]:
def expenses(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df['is_whale'] = 0
    df['is_dolphin'] = 0
    df['is_minnow'] = 0
    df.loc[df['history'] >= 1000, 'is_whale'] = 1
    df.loc[(df['history'] < 1000) & (df['history'] >= 200), 'is_dolphin'] = 1
    df.loc[df['history'] < 200, 'is_minnow'] = 1
    return df

In [ ]:
X_train = expenses(X_train)
X_test = expenses(X_test)

In [ ]:
X_train.sample(5)

__Задание 5__  
провести uplift-моделирование 3 способами: одна модель с признаком коммуникации (S learner), модель с трансформацией таргета (трансформация классов п. 2. 1) и вариант с двумя независимыми моделями  
__Задание 6__  
в конце вывести единую таблицу сравнения метрик uplift@10%, uplift@20% этих 3 моделей


In [ ]:
metrics_df = pd.DataFrame(columns=['uplift@10%', 'uplift@20%'])

In [ ]:
sm = SoloModel(CatBoostClassifier(iterations=20, thread_count=2, random_state=42, silent=True))
sm = sm.fit(X_train, y_train, treat_train, estimator_fit_params={'cat_features': cat_features})

uplift_sm = sm.predict(X_test)
print(uplift_sm)

sm_score_10 = uplift_at_k(y_true=y_test, uplift=uplift_sm, treatment=treat_test, strategy='by_group', k=0.1)
sm_score_20 = uplift_at_k(y_true=y_test, uplift=uplift_sm, treatment=treat_test, strategy='by_group', k=0.2)

metrics_df = metrics_df.append({'uplift@10%': sm_score_10, 'uplift@20%': sm_score_20}, ignore_index=True)

In [ ]:
ct = ClassTransformation(CatBoostClassifier(iterations=20, thread_count=2, random_state=42, silent=True))
ct = ct.fit(X_train, y_train, treat_train, estimator_fit_params={'cat_features': cat_features})

uplift_ct = ct.predict(X_test)
print(uplift_ct)

ct_score_10 = uplift_at_k(y_true=y_test, uplift=uplift_ct, treatment=treat_test, strategy='by_group', k=0.1)
ct_score_20 = uplift_at_k(y_true=y_test, uplift=uplift_ct, treatment=treat_test, strategy='by_group', k=0.2)

metrics_df = metrics_df.append({'uplift@10%': ct_score_10, 'uplift@20%': ct_score_20}, ignore_index=True)

In [ ]:
tm = TwoModels(
    estimator_trmnt=CatBoostClassifier(iterations=20, thread_count=2, random_state=42, silent=True), 
    estimator_ctrl=CatBoostClassifier(iterations=20, thread_count=2, random_state=42, silent=True), 
    method='vanilla'
)
tm = tm.fit(
    X_train, y_train, treat_train,
    estimator_trmnt_fit_params={'cat_features': cat_features}, 
    estimator_ctrl_fit_params={'cat_features': cat_features}
)

uplift_tm = tm.predict(X_test)
print(uplift_tm)

tm_score_10 = uplift_at_k(y_true=y_test, uplift=uplift_tm, treatment=treat_test, strategy='by_group', k=0.1)
tm_score_20 = uplift_at_k(y_true=y_test, uplift=uplift_tm, treatment=treat_test, strategy='by_group', k=0.2)

metrics_df = metrics_df.append({'uplift@10%': tm_score_10, 'uplift@20%': tm_score_20}, ignore_index=True)

In [ ]:
metrics_df.index = ['Solo Learner', 'Transform Learner', 'Two Model Learner']